In [ ]:
import pandas as pd
import os
import librosa.display
import ast
import soundfile as sf
import numpy as np
# Function to perform Voice Activity Detection (VAD)
from utils.Preprocessing import Preprocessing
import json


In [2]:
ROOT_FOLDER = ''
folder_path = os.path.join(ROOT_FOLDER, 'data/Rec/')
audio_files = os.listdir(folder_path)
Transcript_df = pd.read_csv(os.path.join(ROOT_FOLDER, 'data/Transcription_data_from_mihub.csv'))

In [3]:
print(Transcript_df.shape)
print(Transcript_df['audio_name'].nunique())
temp_list = list(Transcript_df['audio_name'])

(956, 7)
956


In [4]:
def chunk_prep(Pre_class, filename,  list_time, list_trans, cat):
    c_time = 0
    c_audio = np.array([])
    c_trans = ""
    silence = np.zeros(int(Pre_class.sr * 0.4), dtype=c_audio.dtype)#400 ms natural pause

    for ind, stamp in enumerate(list_time):

        temp = getattr(Pre_class, cat)[int(stamp[0]*Pre_class.sr):int(stamp[1]*Pre_class.sr)+1]
        temp = Pre_class.voice_activity_detection(temp)
        

        if c_time + (len(temp)/Pre_class.sr) > 29.5:
            segment_path = os.path.join(ROOT_FOLDER,'data/Processed_Files',f"{filename}_{cat}_{ind}.wav")
            sf.write(segment_path, c_audio, Pre_class.sr)
            transcript_json[f"{filename}_{cat}_{ind}.wav"] = c_trans
            
            c_time = (len(temp)/Pre_class.sr)
            c_audio = temp
            c_trans = list_trans[ind]

        else:
            c_time += (len(temp)/Pre_class.sr) + 0.4
            c_audio = np.concatenate((c_audio,silence,temp))
            c_trans += " " + list_trans[ind]
    
    if c_time > 5:
        segment_path = os.path.join(ROOT_FOLDER,'data/Processed_Files',f"{filename}_{cat}_{ind}.wav")
        sf.write(segment_path, c_audio, Pre_class.sr)
        transcript_json[f"{filename}_{cat}_{ind}.wav"] = c_trans
    

In [5]:
cnt_process = 0
transcript_json = {}
for filename in audio_files:
    #print(filename)
    if cnt_process%50 == 0:
        print(cnt_process)
    if filename in temp_list:
        try:
            trans = Transcript_df.loc[Transcript_df['audio_name'] == filename].reset_index().loc[0]
            Pre_class = Preprocessing(os.path.join(ROOT_FOLDER,'data/Rec/',filename), trans=trans)
            Pre_class.separate_channels()
            chunk_prep(Pre_class ,filename,Pre_class.agent_time, Pre_class.agent_trans, "lc")
            chunk_prep(Pre_class ,filename,Pre_class.customer_time, Pre_class.customer_trans, "rc")
            cnt_process += 1
        except Exception as e:
            print(e)
            pass

save_path = os.path.join(ROOT_FOLDER, 'data/Processed_Files', "Transcript.json")
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(transcript_json, f, ensure_ascii=False, indent=4)


0
50
100
150
200
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [6]:
print(cnt_process)

956


In [ ]:
# Read JSON file
with open(os.path.join(ROOT_FOLDER, 'data/Processed_Files', "Transcript.json"), "r", encoding="utf-8") as f:
    transcript_json = json.load(f)

In [ ]:

file = '20241208090513_9880986445.mp3_lc_30.wav'
audio, sr = librosa.load(os.path.join(ROOT_FOLDER,'data/Processed_Files/',file), mono=True, sr=16000)

In [13]:
import torchaudio
import matplotlib.pyplot as plt
from IPython.display import Audio
display(Audio(audio, rate=sr))

In [14]:
transcript_json[file]

'why there the technical issue sir okay do one thing going going the application select the customer amount and select your upi whatever medium you are paying through select the proper account and make the payment there will be know technical issue just right once again yes you can select how are you making the regular payment for your minimum payment just follow the same process and just select the customer amount option yes then can you do the direct bank bank transfer option no problem know problem you can take it as a partial payment no problem'